In [2]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "../figs/pbp_related/"
img_DIR_P = "../data/images/players/"
data_DIR = "../data/pbpdata/"

In [4]:
league = "NBA"
season = "2024"
player_dict = get_players_pbp(league=league)  # Creating Player Dictionary
team_dict = nba_teams.get_teams()  # Creating Team Dictionary
year = season + "-" + str(int(season)+1)[-2:]
with zstd.open(data_DIR + league+"_PBPdata_" + season + ".pkl.zst","rb") as f:
    games_list = dill.load(f)

In [8]:
filepath = data_DIR + league+"_PBPdata_" + season + ".pkl.zst"
tstamp = os.path.getmtime(filepath)

In [12]:
datetime.fromtimestamp(tstamp).strftime('%a %d %b, %Y - %H:%M:%S')

'Thu 16 Jan, 2025 - 06:51:08'

In [ ]:
possessions = [game.possessions.items for game in games_list]
possession_events = list(chain(*[possession.events for possession in list(chain(*possessions))]))

: 

In [ ]:
poss = []
for po in possession_events:
    if (
        isinstance(po, FieldGoal)
        and po.is_blocked
    ):
        try:
            poss.append(po.shot_data)
        except:
            pass

: 

In [ ]:
df1 = pd.DataFrame(poss)
df1["distance"] = np.sqrt(df1["X"]**2 + df1["Y"]**2)/10

: 

In [ ]:
df2 = df1.groupby(["BlockPlayerId"])[["distance"]].agg(["count","mean"])
df2 = df2.reset_index()
df2.columns = ["pID","Blocks","Distance"]
df3 = df2.query("Blocks >=30").sort_values("Distance",ascending=False).reset_index(drop=True)
df3["Player"] = df3["pID"].map(player_dict)
df3

: 

In [ ]:
p = (
    ggplot(df2)
    + aes(x="Blocks",y="Distance")
    + geom_point()
    + theme_idv
)
p

: 

In [ ]:
p = (
    ggplot(df3)
    + aes(x="Blocks",y="Distance",label="Player")
    + geom_point()
    + geom_text(adjust_text={ 'expand_points': (2, 2),'arrowprops': {'arrowstyle': '->','color': 'red'}})
    + theme_idv
)
p

: 

: 